# A full look into `providers` objects

In [ ]:
import xyzservices.providers as xyz

## What is this "provider" object ?

The provider objects are stored as dictionaries (with attribute access for easy access), so we can explore them in Python (or using tab completion in an interactive session or as a collapsible inventory in Jupyter!):

In [ ]:
xyz

Most of those providers have themselves multiple options. For example, OpenStreetMap provides multiple background styles:

In [ ]:
xyz.OpenStreetMap

Looking at a single one of those options, for example "Mapnik" (the default OpenStreetMap background), we can see this is a `TileProvider` object, which behaves as a dict:

In [ ]:
type(xyz.OpenStreetMap.Mapnik)

We can explore its contents:

In [ ]:
xyz.OpenStreetMap.Mapnik

A "provider object" is then a dictionary with a few required entries, such as the url (required, with `{z}/{x}/{y}` to be replaced), with some additional metadata information (optional). These can be the attribution text to be used on the plot (`attribution`), or the maximum zoom level available for these tiles (`max_zoom`). When this information is available, other libraries like `geopandas` or `contextily` may be smart enough to access and use it automatically.

## Which providers are available by default?

The providers definitions that are shipped with `xyzservices` in the `xyzservices.providers` submodule, are coming from the [`leaflet-providers`](https://github.com/leaflet-extras/leaflet-providers) package, an extension to javascript mapping library Leaflet that contains configurations for various free tile providers. Thus, all providers that are listed on their preview [http://leaflet-extras.github.io/leaflet-providers/preview/](http://leaflet-extras.github.io/leaflet-providers/preview/) should also be available in `xyzservices`. On top of that, `xyzservices` can have a few more as a bonus.

```{important}
You should always check the license and terms and conditions of XYZ tiles you want to use. Not all of them can be used in all circumstances.
```

## Specifying options for a provider

Some providers require additional information, such as an API access key. This can be specified by calling the provider object. Any keyword specified will override the default value in the provider object.

For example, the OpenWeatherMap requires an API key:

In [ ]:
xyz.OpenWeatherMap.Clouds

In [ ]:
xyz.OpenWeatherMap.Clouds.requires_token()

We can specify this API key by calling the object or overriding the attribute:

In [ ]:
# Overriding the attribute will alter existing object
xyz.OpenWeatherMap.Clouds["apiKey"] = "my-private-api-key"

# Calling the object will return a copy
xyz.OpenWeatherMap.Clouds(apiKey="my-private-api-key")

This can then be specified where a key or token is expected. For example:
    
```python
contextily.add_basemap(ax, source=xyz.OpenWeatherMap.Clouds(apiKey="my-private-api-key"))
```

```{note}
It may occasionally happen that the `TileProvider` is broken. In such a case, it will have additional attribute `status` with a `"broken"` flag. If you think that the `TileProvider` is broken but it is not flagged as such, please [report it](https://github.com/geopandas/xyzservices/issues).
```


## Building the tile URL

Some packages like `contextily` can use `xyzservices.TileProvider` directly and parse the required information under the hood. Others need and explicit URL string, attribution and other attributes. However, not every package need the URL in the same format. `xyzsevices` can build the URL in most of the required formats, with or without placeholders.

Typical use case would be a folium map, which needs an URL and an attribution with `{x}`, `{y}` and `{z}` placeholders but already filled API keys if needed. You can get the most standard URL in the  `'https://myserver.com/tiles/{z}/{x}/{y}.png'` format via `TileProvider.build_url()`. And if you want a sharper resolution (when supported by the provider), you can pass an optional `scale_factor` attribute.

In [ ]:
import folium

tiles = xyz.CartoDB.Positron

folium.Map(
    location=[53.4108, -2.9358],
    tiles=tiles.build_url(scale_factor="@2x"),
    attr=tiles.html_attribution,
)

If you need an URL of a single tile, you can pass `x`, `y` and `z` directly, alongside other required attributes like `accessToken`.

In [ ]:
xyz.OpenWeatherMap.Clouds.build_url(x=12, y=21, z=15, apiKey="my-private-api-key")

## Overview of built-in providers

Let us create a flat dictionary of the built-in providers:

In [ ]:
providers = xyz.flatten()

This results in quite a few of them already available:

In [ ]:
len(providers)

For this illustration, let's single out the following ones and use `folium` to have a look at them:

In [ ]:
selection = ['OpenStreetMap.Mapnik', 
             'OpenTopoMap', 
             'Stamen.Toner', 
             'Stamen.TonerLite', 
             'Stamen.Terrain', 
             'Stamen.TerrainBackground', 
             'Stamen.Watercolor', 
             'NASAGIBS.ViirsEarthAtNight2012', 
             'CartoDB.Positron', 
             'CartoDB.Voyager'
            ]

You first create a `Map` instance, then looping through the `selection` add all selected base maps as layers. You can switch between them using the layer control in the top-right corner.

In [24]:
m = folium.Map(location=[53.4108, -2.9358], zoom_start=8)

for tiles_name in selection:
    tiles = providers[tiles_name]
    folium.TileLayer(
        tiles=tiles.build_url(),
        attr=tiles.html_attribution,
        name=tiles.name,
    ).add_to(m)

folium.LayerControl().add_to(m)

m

## Loading providers from [Quick Map Services](https://qms.nextgis.com/)

Even though `xyzservices` comes with a large number of built-in providers, you can load even more directly from [Quick Map Services](https://qms.nextgis.com/) using the `TileProvider.from_qms()` method. Note that the name needs to match the name in the QMS database.

In [25]:
from xyzservices import TileProvider

qms_provider = TileProvider.from_qms("OpenTopoMap")
qms_provider

{'name': 'OpenTopoMap',
 'url': 'https://tile.opentopomap.org/{z}/{x}/{y}.png',
 'min_zoom': 0,
 'max_zoom': 19,
 'attribution': 'OpenTopoMap (CC-BY-SA)'}

```{important}
You should always check the license and terms and conditions of XYZ tiles you want to use. Not all of them can be used in all circumstances and not all Quick Map Services can be considered free even though they may work.
```